In [1]:
import pandas as pd
df = pd.read_csv("product_info.csv")

In [2]:
print("Rows, Columns:", df.shape)
print("\nColumns:\n", df.columns.tolist())

Rows, Columns: (8494, 27)

Columns:
 ['product_id', 'product_name', 'brand_id', 'brand_name', 'loves_count', 'rating', 'reviews', 'size', 'variation_type', 'variation_value', 'variation_desc', 'ingredients', 'price_usd', 'value_price_usd', 'sale_price_usd', 'limited_edition', 'new', 'online_only', 'out_of_stock', 'sephora_exclusive', 'highlights', 'primary_category', 'secondary_category', 'tertiary_category', 'child_count', 'child_max_price', 'child_min_price']


In [3]:
print("\nFirst 5 rows:")
display(df.head(5))   


First 5 rows:


,product_id,product_name,brand_id,brand_name,loves_count,rating,reviews,size,variation_type,variation_value,...,online_only,out_of_stock,sephora_exclusive,highlights,primary_category,secondary_category,tertiary_category,child_count,child_max_price,child_min_price
0,P473671,Fragrance Discovery Set,6342,19-69,6320,3.6364,11.0,NaN,NaN,NaN,...,1,0,0,"['Unisex/ Genderless Scent', 'Warm &Spicy Scen...",Fragrance,Value & Gift Sets,Perfume Gift Sets,0,NaN,NaN
1,P473668,La Habana Eau de Parfum,6342,19-69,3827,4.1538,13.0,3.4 oz/ 100 mL,Size + Concentration + Formulation,3.4 oz/ 100 mL,...,1,0,0,"['Unisex/ Genderless Scent', 'Layerable Scent'...",Fragrance,Women,Perfume,2,85.0,30.0
2,P473662,Rainbow Bar Eau de Parfum,6342,19-69,3253,4.2500,16.0,3.4 oz/ 100 mL,Size + Concentration + Formulation,3.4 oz/ 100 mL,...,1,0,0,"['Unisex/ Genderless Scent', 'Layerable Scent'...",Fragrance,Women,Perfume,2,75.0,30.0
3,P473660,Kasbah Eau de Parfum,6342,19-69,3018,4.4762,21.0,3.4 oz/ 100 mL,Size + Concentration + Formulation,3.4 oz/ 100 mL,...,1,0,0,"['Unisex/ Genderless Scent', 'Layerable Scent'...",Fragrance,Women,Perfume,2,75.0,30.0
4,P473658,Purple Haze Eau de Parfum,6342,19-69,2691,3.2308,13.0,3.4 oz/ 100 mL,Size + Concentration + Formulation,3.4 oz/ 100 mL,...,1,0,0,"['Unisex/ Genderless Scent', 'Layerable Scent'...",Fragrance,Women,Perfume,2,75.0,30.0


In [4]:
from collections import Counter
import pandas as pd
import ast

# turning the primary category to lowercase and removing extra space
df['primary_category_clean'] = (
    df['primary_category']
      .astype(str)
      .str.strip()
      .str.lower()
)

# checking for all the classes in primary category
pc_counts = df['primary_category_clean'].value_counts(dropna=False)
print("\nPrimary category value counts:\n", pc_counts)

# putting the highlights in a list with string separated commas
def safe_parse_list(x):
    if pd.isna(x):
        return []
    if isinstance(x, list):
        return x
    if isinstance(x, str):
        x = x.strip()
        # typical list-like string starts with '['
        if x.startswith('[') and x.endswith(']'):
            try:
                parsed = ast.literal_eval(x)
                if isinstance(parsed, (list, tuple)):
                    return list(parsed)
            except Exception:
                pass
        # sometimes highlights stored as comma-separated string
        if ',' in x:
            return [s.strip() for s in x.split(',') if s.strip()]
    return [str(x)]
df['highlights_parsed'] = df['highlights'].apply(safe_parse_list)

# Drop rows where 'highlights_parsed' is empty
df = df[df['highlights_parsed'].apply(lambda x: len(x) > 0)].reset_index(drop=True)

# printing first 10 rows of highlights
sample_highlights = df['highlights_parsed'].head(10).tolist()
print("\nSample parsed highlights (first 10 rows):")
for i, hl in enumerate(sample_highlights, 1):
    print(i, hl)

all_tags = [tag for sub in df['highlights_parsed'] for tag in sub if tag]
tag_counts = Counter(all_tags)
print("\nTop 30 tags in highlights (tag : count):")
for tag, cnt in tag_counts.most_common(30):
    print(f"{tag} : {cnt}")

# keep only skincare as primary_category
skincare_mask = df['primary_category_clean'].str.contains('skincare', na=False)
skincare_df = df[skincare_mask].copy()
print(f"\nKeeping skincare only — rows before: {len(df)}, after filter: {len(skincare_df)}")



Primary category value counts:
 primary_category_clean
skincare           2420
makeup             2369
hair               1464
fragrance          1432
bath & body         405
mini size           288
men                  60
tools & brushes      52
gifts                 4
Name: count, dtype: int64

Sample parsed highlights (first 10 rows):
1 ['Unisex/ Genderless Scent', 'Warm &Spicy Scent', 'Woody & Earthy Scent', 'Fresh Scent']
2 ['Unisex/ Genderless Scent', 'Layerable Scent', 'Warm &Spicy Scent']
3 ['Unisex/ Genderless Scent', 'Layerable Scent', 'Woody & Earthy Scent']
4 ['Unisex/ Genderless Scent', 'Layerable Scent', 'Warm &Spicy Scent']
5 ['Unisex/ Genderless Scent', 'Layerable Scent', 'Woody & Earthy Scent']
6 ['Warm &Spicy Scent', 'Unisex/ Genderless Scent', 'Layerable Scent']
7 ['Unisex/ Genderless Scent', 'Layerable Scent', 'Woody & Earthy Scent']
8 ['Unisex/ Genderless Scent', 'allure 2022 Best of Beauty Award Winner', 'Fresh Scent']
9 ['Fresh Scent', 'Layerable Scent', 'Unisex

In [5]:
# save this to an excel file
skincare_df.to_csv("skincare_only.csv", index=False)
print("Saved filtered file => skincare_only.csv")

Saved filtered file => skincare_only.csv


In [7]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Load dataset
df = pd.read_csv("skincare_only.csv")

# Replace NaNs
df = df.fillna("")

# Normalize numeric features
scaler = MinMaxScaler()
#convert invalid string to 
df['rating'] = pd.to_numeric(df['rating'], errors='coerce').fillna(0)
df['loves_count'] = pd.to_numeric(df['loves_count'], errors='coerce').fillna(0)
df['rating_norm'] = scaler.fit_transform(df[['rating']])
df['popularity_norm'] = scaler.fit_transform(df[['loves_count']])
df['combined_score'] = 0.7 * df['rating_norm'] + 0.3 * df['popularity_norm']

In [8]:
import pandas as pd
import numpy as np
import ast
import re
from collections import Counter

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler

In [9]:
df = pd.read_csv("skincare_only.csv")   
print("Rows, cols:", df.shape)

Rows, cols: (2003, 29)


In [10]:
df['highlights_parsed'] = df['highlights'].apply(safe_parse_list)
# quick check
for i, row in df[['product_id','product_name','highlights_parsed']].head(6).iterrows():
    print(row['product_id'], row['product_name'], row['highlights_parsed'])

P439055 GENIUS Sleeping Collagen Moisturizer ['Vegan', 'Good for: Loss of firmness', 'Collagen', 'Hypoallergenic', 'Without Parabens', 'Best for Dry, Combo, Normal Skin']
P421277 GENIUS Liquid Collagen Serum ['Vegan', 'Good for: Loss of firmness', 'Collagen', 'Hypoallergenic', 'Without Parabens', 'Best for Dry, Combo, Normal Skin']
P432045 GENIUS Liquid Collagen Lip Treatment ['Vegan', 'Good for: Loss of firmness', 'Plumping', 'Collagen', 'Hypoallergenic', 'Best for Dry, Combo, Normal Skin']
P311143 SUBLIME DEFENSE Ultra Lightweight UV Defense Fluid SPF 50 ['Vegan', 'Hypoallergenic', 'UV Protection', 'SPF', 'Best for Dry, Combo, Normal Skin']
P384537 GENIUS Ultimate Anti-Aging Cream ['Vegan', 'Collagen', 'Hypoallergenic', 'Good for: Loss of firmness', 'Best for Dry, Combo, Normal Skin']
P500716 10 Day Results Kit ['Vegan', 'Good for: Loss of firmness', 'Good for: Dark Circles', 'Collagen', 'Hydrating', 'Good for: Anti-Aging']


In [11]:
# replace missing values with empty string and string to lower case(normalization)
df['tertiary_category_clean'] = df['tertiary_category'].fillna('').astype(str).str.strip().str.lower()

In [12]:
#combine primary category, sec category, tertairy category , highlights and ingrediants text
def join_text_field(x):
    if isinstance(x, list):
        return ' '.join([str(t) for t in x])
    return str(x) if pd.notna(x) else ''

df['text_for_tfidf'] = (
    df['primary_category_clean'].fillna('') + ' ' +
    df['secondary_category'].fillna('') + ' ' +
    df['tertiary_category_clean'].fillna('') + ' ' +
    df['highlights_parsed'].apply(join_text_field) + ' ' +
    df['ingredients'].fillna('').astype(str)
).str.replace(r'[\[\]]', ' ', regex=True)   #remove list brackets


In [13]:
print(df['text_for_tfidf'].iloc[0][:400])

skincare Moisturizers moisturizers Vegan Good for: Loss of firmness Collagen Hypoallergenic Without Parabens Best for Dry, Combo, Normal Skin  'Collagen (Vegan)*, Water (Aqua, Eau), Ethylhexyl Palmitate, Oryza Sativa (Rice) Bran Extract, Caprylic/Capric Triglyceride, Glycerin, Cetearyl Methicone, Dimethicone, Cetearyl Alcohol, Pyrus Malus (Apple) Fruit Extract, Chlorella Protothecoides Oil, Polyso


In [14]:
#removes stop words and converts all the text into numerical vectors
tfidf = TfidfVectorizer(stop_words='english', max_features=6000, ngram_range=(1,2))
tfidf_matrix = tfidf.fit_transform(df['text_for_tfidf'])
print("TF-IDF matrix shape:", tfidf_matrix.shape) #shape has n_products, n_features


TF-IDF matrix shape: (2003, 6000)


In [15]:
# user input
user_preferences = ['vegan', 'oily skin', 'spf']
tertiary_category = 'sunscreen' 

In [16]:
def expand_keywords(keywords):
    # join all the user prefernce words
    return [k.lower() for k in keywords]

expanded = expand_keywords(user_preferences)
query_text = ' '.join(expanded)
if tertiary_category:
    query_text += ' ' + str(tertiary_category).lower()

In [17]:
# vectorize user input
user_vec = tfidf.transform([query_text])  

In [18]:
# Measures the similarity between the user input and the dataset
from sklearn.metrics.pairwise import cosine_similarity

sim_scores = cosine_similarity(user_vec, tfidf_matrix).flatten()  # shape: (n_products,)

In [19]:
# map keywords with the user input
keyword_map = {
    'hydrating': ['hydrating', 'moisturiz', 'moisturizer', 'hydration', 'hydrating'],
    'moisturizer': ['moisturizer', 'moisturising', 'moisturizing'],
    'sunscreen': ['sunscreen', 'spf', 'sunblock', 'sun protection'],
    'anti-aging': ['anti-aging', 'anti age', 'age-defying', 'firming'],
    'acne': ['acne', 'blemish', 'breakout', 'anti-acne'],
    'oily skin': ['oily skin', 'for oily', 'matte'],
    'dry skin': ['dry skin', 'for dry'],
    'sensitive skin': ['sensitive', 'hypoallergenic', 'dermatologist'],
    'vegan': ['vegan', 'cruelty-free']
}

def expand_keywords(user_terms):
    expanded = []
    lowered = [t.lower() for t in user_terms]
    for term in lowered:
        matched = False
        for k, variants in keyword_map.items():
            if term in variants or k == term:
                expanded.extend(variants)
                matched = True
                break
        if not matched:
            expanded.append(term)
    # dedupe preserving order
    seen = set()
    out = []
    for t in expanded:
        if t not in seen:
            out.append(t)
            seen.add(t)
    return out

In [20]:
df['rating_norm'] = scaler.fit_transform(df[['rating']])
df['popularity_norm'] = scaler.fit_transform(df[['loves_count']])
df['combined_score'] = 0.7 * df['rating_norm'] + 0.3 * df['popularity_norm']

sim_weight = 0.7
score_weight = 0.3

df['sim_score'] = sim_scores
df['final_score'] = sim_weight * df['sim_score'] + score_weight * df['combined_score']

In [21]:
candidates = df.copy()

price_min = 0       # or None if you don't want a lower limit
price_max = 50      # max price filter
tertiary_category = 'sunscreen'  # e.g., sunscreen

# Tertiary category filter
if tertiary_category:
    candidates = candidates[
        candidates['tertiary_category_clean'].str.contains(tertiary_category.lower(), na=False)
    ]

# user budget
if price_min is not None:
    candidates = candidates[candidates['price_usd'] >= price_min]
if price_max is not None:
    candidates = candidates[candidates['price_usd'] <= price_max]

In [22]:
top_n = 8
recommendations = candidates.sort_values('final_score', ascending=False).head(top_n)

# Return relevant columns
recommendations = recommendations[
    ['product_name', 'rating', 'loves_count', 'price_usd', 'combined_score', 'final_score']
]

print(recommendations[['product_name', 'rating', 'price_usd', 'final_score']].head(3))

                                           product_name  rating  price_usd  \
243   Full Spectrum 360º Refreshing Water Cream Orga...  4.5805       48.0   
1690         Mineral Sheerscreen Sunscreen SPF 30 PA+++  3.9473       38.0   
1691    Mini Mineral Sheerscreen Sunscreen SPF 30 PA+++  3.9473       22.0   

      final_score  
243      0.506453  
1690     0.316841  
1691     0.315417  


In [23]:
import joblib

# Save TF-IDF vectorizer and matrix
joblib.dump(tfidf, "tfidf_vectorizer.pkl")
joblib.dump(tfidf_matrix, "tfidf_matrix.pkl")

# Save the MinMaxScaler
joblib.dump(scaler, "scaler.pkl")

print("✅ TF-IDF vectorizer, matrix, and scaler saved successfully!")

✅ TF-IDF vectorizer, matrix, and scaler saved successfully!


In [24]:
joblib.dump(df, "skincare_df.pkl")

['skincare_df.pkl']

In [25]:
import joblib
df = joblib.load("skincare_df.pkl")
print(df.columns)

Index(['product_id', 'product_name', 'brand_id', 'brand_name', 'loves_count',
       'rating', 'reviews', 'size', 'variation_type', 'variation_value',
       'variation_desc', 'ingredients', 'price_usd', 'value_price_usd',
       'sale_price_usd', 'limited_edition', 'new', 'online_only',
       'out_of_stock', 'sephora_exclusive', 'highlights', 'primary_category',
       'secondary_category', 'tertiary_category', 'child_count',
       'child_max_price', 'child_min_price', 'primary_category_clean',
       'highlights_parsed', 'tertiary_category_clean', 'text_for_tfidf',
       'rating_norm', 'popularity_norm', 'combined_score', 'sim_score',
       'final_score'],
      dtype='object')


Sentence transformer

In [26]:
# run in a notebook cell (prefix !), or in terminal
!pip install -U sentence-transformers faiss-cpu joblib tqdm
# If you have GPU and want faiss-gpu, install faiss-gpu accordingly (not shown here).

  Attempting uninstall: joblib
    Found existing installation: joblib 1.4.2
    Uninstalling joblib-1.4.2:
      Successfully uninstalled joblib-1.4.2


You should consider upgrading via the 'C:\Users\prana\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [4]:
import os
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import joblib
from tqdm.auto import tqdm

# If you want to use FAISS
import faiss

# Choose device: "cpu" or "cuda" if available
DEVICE = "cpu"  # change to "cuda" if you have GPU

In [5]:
import pandas as pd
import joblib
import re

# Load your data
df = pd.read_csv("skincare_only.csv")

print("Columns found in CSV:", list(df.columns))  # 👀 helps confirm what’s inside

# Define helper to safely get columns
def safe_get(col):
    return df[col].fillna('').astype(str) if col in df.columns else ''

# Build unified text column for embeddings/TF-IDF
df['text_for_tfidf'] = (
    safe_get('primary_category_clean') + ' ' +
    safe_get('secondary_category') + ' ' +
    safe_get('tertiary_category_clean') + ' ' +
    safe_get('tertiary_category') + ' ' +   # backup if clean one missing
    safe_get('highlights_parsed') + ' ' +
    safe_get('ingredients')
).str.replace(r'[\[\]]', ' ', regex=True)

print("✅ Text column created successfully!")
print("Number of products:", len(df))
print(df[['product_name', 'text_for_tfidf']].head(2))

# Save for later use
joblib.dump(df, "skincare_df.pkl")
print("✅ DataFrame saved successfully as skincare_df.pkl")


Columns found in CSV: ['product_id', 'product_name', 'brand_id', 'brand_name', 'loves_count', 'rating', 'reviews', 'size', 'variation_type', 'variation_value', 'variation_desc', 'ingredients', 'price_usd', 'value_price_usd', 'sale_price_usd', 'limited_edition', 'new', 'online_only', 'out_of_stock', 'sephora_exclusive', 'highlights', 'primary_category', 'secondary_category', 'tertiary_category', 'child_count', 'child_max_price', 'child_min_price', 'primary_category_clean', 'highlights_parsed']
✅ Text column created successfully!
Number of products: 2003
                           product_name  \
0  GENIUS Sleeping Collagen Moisturizer   
1          GENIUS Liquid Collagen Serum   

                                      text_for_tfidf  
0  skincare Moisturizers  Moisturizers  'Vegan', ...  
1  skincare Treatments  Face Serums  'Vegan', 'Go...  
✅ DataFrame saved successfully as skincare_df.pkl


In [6]:
MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"
model = SentenceTransformer(MODEL_NAME, device=DEVICE)
model.max_seq_length = 512  # safe default

In [7]:
# When you have many rows, use batching to avoid OOM
texts = df['text_for_tfidf'].astype(str).tolist()
batch_size = 64
embeddings = []

for i in tqdm(range(0, len(texts), batch_size), desc="Encoding"):
    batch_texts = texts[i:i+batch_size]
    emb = model.encode(batch_texts, convert_to_numpy=True, show_progress_bar=False, device=DEVICE)
    embeddings.append(emb)

embeddings = np.vstack(embeddings)  # shape: (n_products, dim)

# Normalize vectors for cosine search (faiss inner-product on L2-normalized dims)
faiss.normalize_L2(embeddings)

print("Embeddings shape:", embeddings.shape)
# Save embeddings array
np.save("product_embeddings.npy", embeddings)
joblib.dump(embeddings, "product_embeddings.pkl")
# Save df and model artifacts
joblib.dump(df, "skincare_df.pkl")
joblib.dump(model, "sbert_model.pkl")  # optional but useful


Encoding:   0%|          | 0/32 [00:00<?, ?it/s]

Embeddings shape: (2003, 384)


['sbert_model.pkl']

In [8]:
from sentence_transformers import SentenceTransformer
import pandas as pd
import numpy as np
import joblib
import faiss
from tqdm.auto import tqdm

# Load your dataset
df = joblib.load("skincare_df.pkl")

# Combine all relevant text columns into one
df["combined_text"] = (
    df["primary_category"].fillna('') + ' ' +
    df["tertiary_category"].fillna('') + ' ' +
    df["highlights_parsed"].astype(str).fillna('') + ' ' +
    df["ingredients"].astype(str).fillna('')
)

# Load Sentence Transformer
model = SentenceTransformer("all-MiniLM-L6-v2")

# Encode the dataset
embeddings = model.encode(df["combined_text"].tolist(), show_progress_bar=True, convert_to_numpy=True)

# Save FAISS index
dim = embeddings.shape[1]
index = faiss.IndexFlatIP(dim)  # Inner product (for cosine similarity)
faiss.normalize_L2(embeddings)
index.add(embeddings)

# Save all artifacts
faiss.write_index(index, "skincare_faiss.index")
joblib.dump(df, "skincare_df.pkl")
model.save("sentence_model")

print("✅ SentenceTransformer embeddings & FAISS index saved successfully!")


Batches:   0%|          | 0/63 [00:00<?, ?it/s]

✅ SentenceTransformer embeddings & FAISS index saved successfully!


In [6]:
# Embeddings must be float32
embeddings = embeddings.astype('float32')
d = embeddings.shape[1]

# We'll use IndexFlatIP (inner product) on normalized vectors -> equivalent to cosine
index = faiss.IndexFlatIP(d)
index.add(embeddings)  # add vectors

# Save index to disk
faiss.write_index(index, "faiss_index.idx")

# Quick sanity check: search top-5 for the first product
D, I = index.search(embeddings[:1], k=5)
print("Distances:", D)
print("Indices:", I)

Distances: [[1.         0.93845505 0.9353634  0.9339823  0.92380285]]
Indices: [[   0    4    9 1564  133]]


In [7]:
def semantic_search(query_text, top_k=10):
    """Return top_k product indices sorted by semantic similarity to query_text."""
    if not query_text or str(query_text).strip()=="":
        return []
    q_emb = model.encode([query_text], convert_to_numpy=True, show_progress_bar=False, device=DEVICE)
    faiss.normalize_L2(q_emb)
    q_emb = q_emb.astype('float32')
    # load index if not in memory
    global index
    if 'index' not in globals():
        index = faiss.read_index("faiss_index.idx")
    D, I = index.search(q_emb, k=top_k)
    # D are inner product scores (cosine since normalized)
    return I[0].tolist(), D[0].tolist()

# Example
inds, scores = semantic_search("vegan sunscreen without parabens for oily skin", top_k=5)
print("Top indices:", inds)
print(df.iloc[inds][['product_name','rating']])


Top indices: [1887, 1906, 1874, 1899, 1237]
                                           product_name  rating
1887       100% Organic Cold-Pressed Moroccan Argan Oil  4.1709
1906                  100% Organic Virgin Chia Seed Oil  4.1892
1874        100% Organic Cold-Pressed Rose Hip Seed Oil  4.3988
1899  Ascorbyl Tetraisopalmitate Solution 20% in Vit...  3.8193
1237                      GO OFF Makeup Dissolving Mist  4.1746


In [8]:
# Load artifacts if needed
embeddings = joblib.load("product_embeddings.pkl")  # shape (n, dim)
df = joblib.load("skincare_df.pkl")
# ensure df index aligns with embeddings order (we used df order when computing)
assert embeddings.shape[0] == len(df)

def get_hybrid_recommendations(query, top_k=50, final_top_n=8, sim_weight=0.7, score_weight=0.3):
    # Step 1: semantic top_k using FAISS
    inds, sim_scores = semantic_search(query, top_k=top_k)
    
    # Convert to DataFrame slice
    candidates = df.iloc[inds].copy().reset_index(drop=False)  # keep original index in 'index'
    # sim_scores are cosine-like in D
    candidates['sim_score'] = sim_scores
    
    # Step 2: normalize sim_score to 0-1
    # sim_scores are cosine (range [-1,1]) but here positive due to normalization
    # scale to 0-1:
    min_s, max_s = candidates['sim_score'].min(), candidates['sim_score'].max()
    if max_s - min_s > 1e-6:
        candidates['sim_norm'] = (candidates['sim_score'] - min_s) / (max_s - min_s)
    else:
        candidates['sim_norm'] = 1.0
    
    # Step 3: ensure we have combined_score in df
    if 'combined_score' not in candidates.columns:
        # fallback: use rating_norm if exist
        candidates['combined_score'] = candidates.get('combined_score', 0)
    
    # Step 4: hybrid final score
    candidates['final_score'] = sim_weight * candidates['sim_norm'] + score_weight * candidates['combined_score']
    
    # Sort and return top final_top_n
    top = candidates.sort_values('final_score', ascending=False).head(final_top_n)
    return top

# Test
top = get_hybrid_recommendations("vegan sunscreen without parabens for oily skin", top_k=100, final_top_n=5)
top[['product_name','sim_score','sim_norm','combined_score','final_score']].head()


,product_name,sim_score,sim_norm,combined_score,final_score
0,100% Organic Cold-Pressed Moroccan Argan Oil,0.631416,1.000000,0,0.700000
1,100% Organic Virgin Chia Seed Oil,0.629203,0.981909,0,0.687336
2,100% Organic Cold-Pressed Rose Hip Seed Oil,0.615686,0.871433,0,0.610003
3,Ascorbyl Tetraisopalmitate Solution 20% in Vit...,0.615554,0.870357,0,0.609250
4,GO OFF Makeup Dissolving Mist,0.608894,0.815927,0,0.571149


In [9]:
# Save model, index, and any arrays that app will load
joblib.dump(model, "sbert_model.pkl")            # optional, but can load the SBERT model in app (heavy)
joblib.dump(df, "skincare_df.pkl")
joblib.dump(embeddings, "product_embeddings.pkl")
faiss.write_index(index, "faiss_index.idx")

In [2]:
import joblib
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# ==============================
# 🔹 Load saved files
# ==============================
tfidf = joblib.load("tfidf_vectorizer.pkl")
tfidf_matrix = joblib.load("tfidf_matrix.pkl")
df = joblib.load("skincare_df.pkl")

print("✅ Files loaded successfully!")
print("Dataset size:", df.shape)
print(df.columns)

# ==============================
# 🔹 Example user query
# ==============================
query = "oil-free sunscreen for acne-prone skin"

# Convert query into TF-IDF vector
query_vec = tfidf.transform([query])

# Compute cosine similarity between query and all products
cosine_sim = cosine_similarity(query_vec, tfidf_matrix)[0]

# Attach similarity scores to dataframe
df['similarity_score'] = cosine_sim

# Sort by similarity score (descending)
top_results = df.sort_values(by='similarity_score', ascending=False).head(3)

# Display the top 5 product matches
print("\n🔍 Query:", query)
print("\nTop 3 matching products:\n")
print(top_results[['product_name', 'primary_category', 'tertiary_category', 'price_usd', 'rating', 'similarity_score']])


✅ Files loaded successfully!
Dataset size: (2003, 30)
Index(['product_id', 'product_name', 'brand_id', 'brand_name', 'loves_count',
       'rating', 'reviews', 'size', 'variation_type', 'variation_value',
       'variation_desc', 'ingredients', 'price_usd', 'value_price_usd',
       'sale_price_usd', 'limited_edition', 'new', 'online_only',
       'out_of_stock', 'sephora_exclusive', 'highlights', 'primary_category',
       'secondary_category', 'tertiary_category', 'child_count',
       'child_max_price', 'child_min_price', 'primary_category_clean',
       'highlights_parsed', 'text_for_tfidf'],
      dtype='object')

🔍 Query: oil-free sunscreen for acne-prone skin

Top 3 matching products:

                                           product_name primary_category  \
243   Full Spectrum 360º Refreshing Water Cream Orga...         Skincare   
1708  PLAY Antioxidant Body Sunscreen Mist SPF 30 PA...         Skincare   
159     Isle Body Oil SPF 30 with Sea Moss and Squalane         Skinca

In [1]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, f1_score

model = SentenceTransformer("all-MiniLM-L6-v2")
df = pd.read_csv("skincare_only.csv")

# Prepare text and categories
df['text'] = df['primary_category'].fillna('') + " " + df['highlights_parsed'].fillna('')
sample_df = df.sample(100, random_state=42).reset_index(drop=True)

# Generate embeddings
embeddings = model.encode(sample_df['text'].tolist(), convert_to_tensor=True)
sim_matrix = util.cos_sim(embeddings, embeddings).cpu().numpy()

# Build pairs
true_labels = []
pred_labels = []

threshold = 0.6  # similarity threshold for "same category"

for i in range(len(sample_df)):
    for j in range(i + 1, len(sample_df)):
        same_cat = sample_df.loc[i, 'primary_category'] == sample_df.loc[j, 'primary_category']
        true_labels.append(1 if same_cat else 0)
        pred_labels.append(1 if sim_matrix[i, j] >= threshold else 0)

# Compute metrics
acc = accuracy_score(true_labels, pred_labels)
f1 = f1_score(true_labels, pred_labels)

print(f"✅ Approx Accuracy: {acc:.3f}")
print(f"✅ Approx F1 Score: {f1:.3f}")


✅ Approx Accuracy: 0.950
✅ Approx F1 Score: 0.974
